In [51]:
import tensorflow as tf 
import string 
import re


In [52]:
text_file = 'spa-eng/spa.txt'
with open (text_file )as f :
    lines = f.read().split('\n')[:-1]
text_pairs = []
for line in lines :
    english, spanish = line.split('\t')
    spanish = '[start]'+ spanish + '[end]'
    text_pairs.append((english,spanish))

In [53]:
import random 
random.shuffle(text_pairs)
num_val_samples = int(0.15 * len(text_pairs))
num_train_samples = len(text_pairs) - 2* num_val_samples
train_pairs = text_pairs[:num_train_samples]
val_pairs = text_pairs[num_train_samples:num_train_samples+num_val_samples]
test_pairs = text_pairs[num_train_samples+num_val_samples:]


In [54]:
strip_chars = string.punctuation +'¿¡'
strip_chars = strip_chars.replace('[','')
strip_chars = strip_chars.replace(']','')

def custom_standardization( input_string):
    lowercase = tf.strings.lower(input_string)
    return tf.strings.regex_replace(lowercase, f'[{re.escape(strip_chars)}]','')

In [55]:
from tensorflow import keras
from keras import layers

In [56]:
vocab_size = 15000
sequence_length = 20
source_vectorization = layers.TextVectorization(
    max_tokens=vocab_size,
    output_mode="int",
    output_sequence_length=sequence_length,
)
target_vectorization = layers.TextVectorization(
    max_tokens=vocab_size,
    output_mode="int",
    output_sequence_length=sequence_length + 1,
    standardize=custom_standardization,
)
train_english_texts = [pair[0] for pair in train_pairs]
train_spanish_texts = [pair[1] for pair in train_pairs]
source_vectorization.adapt(train_english_texts)
target_vectorization.adapt(train_spanish_texts)

2023-09-02 16:47:40.619910: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.
2023-09-02 16:47:43.795389: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.


In [57]:
class TransformerBlock(layers.Layer):
    def __init__(self, embed_dim, num_heads, dense_dim):
        super().__init__()
        self.embed_dim = embed_dim
        self.dense_dim = dense_dim
        self.num_heads = num_heads

        self.att = layers.MultiHeadAttention(num_heads=num_heads, key_dim=embed_dim)
        self.ffn = keras.Sequential([layers.Dense(dense_dim,activation='relu'), layers.Dense(embed_dim),])
        self.layernorm1 = layers.LayerNormalization(epsilon=1e-6)
        self.layernorm2 = layers.LayerNormalization(epsilon=1e-6)
        

    def call(self, inputs, mask=None):
        if mask is not None :
            mask = mask[:,tf.newaxis, :]
        attn_output = self.att(inputs,inputs,attention_mask=mask)
        out1 = self.layernorm1(inputs+attn_output)
        ffn_output = self.ffn(out1)
        return self.layernorm2(out1 + ffn_output)
        
    def get_config(self):
        config = super().get_config()
        config.update({
            'embed_dim':self.embed_dim,
            'num_heads' :self.num_heads,
            'dense_dim':self.dense_dim ,      
            })
        return config

In [58]:
class TransformerDecoder (layers.Layer) :
    def __init__ (self, embed_dim, dense_dim, num_heads, **kwargs) :
        super().__init__(**kwargs)
        self.embed_dim = embed_dim
        self.dense_dim  = dense_dim
        self.num_heads = num_heads
        self.attention_1 = layers.MultiHeadAttention(num_heads=num_heads, key_dim=embed_dim)
        self.attention_2 = layers.MultiHeadAttention(num_heads=num_heads, key_dim=embed_dim)
        self.dense_proj = keras.Sequential(
            [layers.Dense(dense_dim, activation='relu'),
             layers.Dense(embed_dim),]
             )
        self.layernorm_1 = layers.LayerNormalization()
        self.layernorm_2 = layers.LayerNormalization()
        self.layernorm_3 = layers.LayerNormalization()
        self.supports_masking = True

    def get_config (self):
        config = super().get_config()
        config.update({
            'embed_dim': self.embed_dim,
            'num_heads': self.num_heads,
            'dense_dim': self.dense_dim
        })
        return config
    def get_causal_attention_mask(self, inputs) :
        inputs_shape = tf.shape(inputs)
        batch_size, sequence_length = inputs_shape[0], inputs_shape[1]
        i = tf.range(sequence_length)[:,tf.newaxis]
        j = tf.range(sequence_length)
        mask = tf.cast(i >= j, dtype='int32')
        mask = tf.reshape(mask, (1, inputs_shape[1], inputs_shape[1]))
        mult = tf.concat(
            [tf.expand_dims(batch_size, -1),tf.constant([1,1], dtype=tf.int32),],axis=0
        )
        return tf.tile(mask, mult)
    
    def call (self, inputs, encoder_outputs, mask=None):
        causal_mask = self.get_causal_attention_mask(inputs)
        if mask is not None:
            padding_mask = tf.cast(mask[:,tf.newaxis, :], dtype='int32')
            padding_mask = tf.minimum(padding_mask, causal_mask)
        attemtion_output_1 = self.attention_1(query=inputs,value=inputs,key=encoder_outputs,attention_mask=padding_mask)
        attention_output_1 = self.layernorm_1(inputs + attemtion_output_1) 
        attention_output_2 = self.attention_2(query=attention_output_1,value=encoder_outputs,key=encoder_outputs,attention_mask=padding_mask)
        attention_output_2 = self.layernorm_2(attention_output_1 + attention_output_2)
        proj_output = self.dense_proj(attention_output_2)
        return self.layernorm_3(attention_output_2 + proj_output)


        

In [59]:
class TokenAndPositionEmbedding(layers.Layer):
    def __init__(self, maxlen, vocab_size, embed_dim):
        super().__init__()
        self.token_emb = layers.Embedding(input_dim=vocab_size,output_dim=embed_dim)
        self.pos_emb = layers.Embedding(input_dim=maxlen,output_dim=embed_dim)
    def compute_mask(self, inputs, mask=None):
        return tf.math.not_equal(inputs, 0)

    def call(self, x):
        maxlen = tf.shape(x)[-1]
        positions = tf.range(start=0, limit=maxlen, delta=1)
        positions = self.pos_emb(positions)
        x = self.token_emb(x)
        return x + positions

In [60]:

sequence_length = 20
embed_dim = 256
num_heads = 8
dense_dim = 2048

encoder_inputs = keras.Input(shape=(None,), dtype='int64', name='english')
x = TokenAndPositionEmbedding(sequence_length, vocab_size, embed_dim) (encoder_inputs)
encoder_outputs = TransformerBlock(embed_dim, dense_dim, num_heads) (x)

decoder_inputs = keras.Input(shape=(None,), dtype='int64', name='spanish')
x = TokenAndPositionEmbedding(sequence_length, vocab_size, embed_dim) (decoder_inputs)
x = TransformerDecoder(embed_dim, dense_dim, num_heads) (x, encoder_outputs)
x = layers.Dropout(0.5) (x)
decoder_outputs = layers.Dense(vocab_size, activation='softmax') (x)
transformer = keras.Model([encoder_inputs, decoder_inputs], decoder_outputs)

In [ ]:
batch_size = 64

def format_dataset(eng, spa):
    eng = source_vectorization(eng)
    spa = target_vectorization(spa)
    return ({
        "english": eng,
        "spanish": spa[:, :-1],
    }, spa[:, 1:])

def make_dataset(pairs):
    eng_texts, spa_texts = zip(*pairs)
    eng_texts = list(eng_texts)
    spa_texts = list(spa_texts)
    dataset = tf.data.Dataset.from_tensor_slices((eng_texts, spa_texts))
    dataset = dataset.batch(batch_size)
    dataset = dataset.map(format_dataset, num_parallel_calls=4)
    return dataset.shuffle(2048).prefetch(16).cache()

train_ds = make_dataset(train_pairs)
val_ds = make_dataset(val_pairs)

In [62]:
transformer.compile(optimizer='rmsprop',loss='sparse_categorical_crossentropy',metrics=['accuracy'])
transformer.fit(train_ds,epochs=10, validation_data=val_ds)

Epoch 1/10


2023-09-02 16:47:51.865774: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.


   1/1302 [..............................] - ETA: 46:48:14 - loss: 3.0828 - accuracy: 0.0000e+00